# IterativeEikonal
Approximate the viscosity solution of the  Eikonal equation 
$$\begin{cases} \Vert \nabla_G W(g) \Vert = 1, & g \in G, \\
W(g) = 0, & g \in S \subset G. \end{cases}$$

In [ ]:
import iterativeeikonal as eik
import taichi as ti
ti.init(arch=ti.gpu, debug=False)
import numpy as np
import scipy as sp
import diplib as dip
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib widget

## $\mathbb{R}^2$

### Flat space
In fact, we will solve the Eikonal equation on some subset $G \subset \mathbb{R}^2$, namely $G = [-1, 1] \times [-1, 1]$. The source set will consist of a single point: $S = \{(0, 0)\}$. Then we know that the viscosity solution is simply the Euclidean norm.

In [ ]:
def iterative_method_flat_R2(N, n):
    """Discretise [-1, 1] x [-1, 1] into `N` points in each direction, and apply the iterative solution method `n` times."""
    dxy = 2. / (N + 1)
    eps = dxy / 4

    W = get_initial_W(N)
    boundarypoints, boundaryvalues = get_boundary_conditions(N)

    eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)

    dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy = get_initial_derivatives(W)
    for _ in range(n):
        step_W(W, dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy, dxy, eps)
        eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)

    W_np = W.to_numpy()
    return eik.cleanarrays.unpad_array(W_np)

def get_initial_W(N):
    W_unpadded = np.full(shape=(N, N), fill_value=100.)
    W_np = eik.cleanarrays.pad_array(W_unpadded, pad_value=100., pad_shape=1)
    W = ti.field(dtype=ti.f32, shape=W_np.shape)
    W.from_numpy(W_np)
    return W

def get_boundary_conditions(N):
    i_0, j_0 = (N + 1) // 2, (N + 1) // 2
    boundarypoints_np = np.array([[i_0, j_0]], dtype=int)
    boundaryvalues_np = np.array([0.], dtype=float)
    boundarypoints = ti.Vector.field(n=2, dtype=ti.i32, shape=1)
    boundarypoints.from_numpy(boundarypoints_np)
    boundaryvalues = ti.field(shape=1, dtype=ti.f32)
    boundaryvalues.from_numpy(boundaryvalues_np)
    return boundarypoints, boundaryvalues

def get_initial_derivatives(W):
    dx_forward = ti.field(dtype=ti.f32, shape=W.shape)
    dx_backward = ti.field(dtype=ti.f32, shape=W.shape)
    dy_forward = ti.field(dtype=ti.f32, shape=W.shape)
    dy_backward = ti.field(dtype=ti.f32, shape=W.shape)
    abs_dx = ti.field(dtype=ti.f32, shape=W.shape)
    abs_dy = ti.field(dtype=ti.f32, shape=W.shape)
    return dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy

@ti.kernel
def step_W(
    W:ti.template(), 
    dx_forward: ti.template(), 
    dx_backward: ti.template(), 
    dy_forward: ti.template(), 
    dy_backward: ti.template(), 
    abs_dx: ti.template(), 
    abs_dy: ti.template(), 
    dxy: ti.f32, 
    eps: ti.f32
):
    eik.derivativesR2.abs_derivatives(W, dxy, dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy)
    for I in ti.grouped(W):
        W[I] += (1 - ti.math.sqrt(abs_dx[I] ** 2 + abs_dy[I] ** 2)) * eps

In [ ]:
N_small = 11
n_small = 50
xs_small, ys_small = np.meshgrid(np.linspace(-1, 1, N_small), np.linspace(-1, 1, N_small))
W_iterative_small = iterative_method_flat_R2(N_small, n_small)

N_big = 51
n_big = 250
xs_big, ys_big = np.meshgrid(np.linspace(-1, 1, N_big), np.linspace(-1, 1, N_big))
W_exact = np.sqrt(xs_big ** 2 + ys_big ** 2)
W_iterative_big = iterative_method_flat_R2(N_big, n_big)

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
contour = ax.contour(xs_small, ys_small, W_iterative_small, linestyles="dotted")
contour = ax.contour(xs_big, ys_big, W_iterative_big, linestyles="dashed")
ax.contour(xs_big, ys_big, W_exact)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_xlabel("x")
ax.set_ylabel("y")
fig.colorbar(contour, label="W(x, y)");

### Retinal image
Computing the cost function on flat $\mathbb{R}^2$ is not very impressive. We would like to be able to do it also when the space is not flat, for instance in the case of a retinal image. To track the vessels in a retinal image, it makes sense to assign a low cost to those areas of the image that are vessels, and a high cost to those that aren't. We must therefore first compute such a cost function.

To do this, we use Frangi vesselness filtering.

#### Frangi vesselness filtering

In [ ]:
ds = 8
retinal_image = Image.open("E46_OD_best.tif")
width, height = retinal_image.size
retinal_image_gray_ds = retinal_image.resize((width // ds, height // ds)).convert("L")
retinal_array_unnormalised = np.array(retinal_image_gray_ds).astype(np.float64)
retinal_array = retinal_array_unnormalised / retinal_array_unnormalised.max()

Frangi vesselness filtering uses Gaussian derivatives. It turns out that the standard function for computing Gaussian derivatives in SciPy - `scipy.ndimage.gaussian_filter()` - is pretty inaccurate, especially at small scales (the derivative is much sharper). We therefore make use of the Gaussian derivatives defined in another package: DIPlib. 

In [ ]:
Lxx_sp = sp.ndimage.gaussian_filter(-retinal_array, sigma=0.25, order=(0, 2))
print(f"Lxx[SciPy] in [{Lxx_sp.min()}, {Lxx_sp.max()}]")
Lxx_sp_shift = Lxx_sp - Lxx_sp.min()
Lxx_sp = Lxx_sp_shift / Lxx_sp_shift.max()
Lxx_dip = np.array(dip.Gauss(-retinal_array, (0.25, 0.25), (2, 0)))
print(f"Lxx[DIPlib] in [{Lxx_dip.min()}, {Lxx_dip.max()}]")
Lxx_dip_shift = Lxx_dip - Lxx_dip.min()
Lxx_dip = Lxx_dip_shift / Lxx_dip_shift.max()

In [ ]:
_, fig = eik.cleanarrays.view_image_arrays_side_by_side((Lxx_sp, Lxx_dip))

In [ ]:
scales = np.array((0.25, 0.5, 1, 2, 3, 4), dtype=float) # / ds
vesselness = eik.costfunctions.multiscale_frangi_filter_R2(-retinal_array, 0.3, 0.3, scales)
mask = (retinal_array > 0) # Remove boundary
vesselness *= sp.ndimage.binary_erosion(mask, iterations=4)
print(f"Vesselness in [{vesselness.min()}, {vesselness.max()}]")
vesselness /= vesselness.max()

In [ ]:
image_vesselness, fig = eik.cleanarrays.view_image_array(vesselness)

Compared to the result produced by the Frangi filter in the Mathematica notebook "code A - Vesselness in SE(2).nb", this vesselness score is much sharper.

#### Cost function
Given the vesselness function, we compute the cost function as
$$\mathrm{cost}_{\lambda, p}(x, y) = \frac{1}{1 + \lambda * \vert \mathrm{vesselness} \vert^p}$$

In [ ]:
cost = eik.costfunctions.cost_function(vesselness, 1000, 3)
_, fig = eik.cleanarrays.view_image_array(cost)